In [96]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier

import scipy.stats as scs
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, precision_score, accuracy_score, recall_score, f1_score



def eval(model, X, y):
    y_predicted = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y,y_predicted))
    print(y)
    print(y_predicted)
    #precision = precision_score(y,y_prdict)
    
    y_base = np.ones(len(y))*np.average(y)
    rmse_base = np.sqrt(mean_squared_error(y,y_base))
    
    print(model)
    
    print('R2 of model: {:2.2f}'.format(r2_score(y,y_predicted)))
    
    print('RMSE of model: {:2.2f}'.format(mean_squared_error(y,y_predicted)))
    print('Base RMSE: {}'.format(rmse_base))
    
    print('Normalized RSME (RMSE of predict/RMSE of base) : {:2.2f}'.format(rmse/rmse_base))

    return rmse

In [78]:
#from twilio.rest import Client
#accountSID = 'AC54b5e4924329b419847933461aa3c8ce'
#authToken = '499e9beddbf0fd7db7f2f42c92eba3c4'
#client = Client(accountSID, authToken)
#myTwilioNumber = '+12254522380'
#myCellPhone = '+16507030878'

In [79]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(2051,input_dim=2051, activation='relu'))
    model.add(Dense(2000, activation='relu'))
    model.add(Dense(1, activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model


In [80]:
all_data = pd.read_csv('comb_all_data.csv')
print(len(all_data))
no_dups = all_data.drop_duplicates()
no_nan = no_dups.dropna()
print(len(no_dups))
print(len(no_nan))

config = no_nan.head(20000)
print(len(config))

1184606
1184606
1184606
20000


In [81]:
dummies = config.select_dtypes('object').columns
y = config['Price']
magic = config.drop(columns =['Price'])
X = pd.get_dummies(magic, columns = dummies, dummy_na = True, prefix=dummies)

X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.20, random_state = 10)
X_train.shape

(16000, 1090)

In [82]:
X_train

,Milage,Age,Make_AC,Make_Acura,Make_Alfa Romeo,Make_Am General,Make_Aston Martin,Make_Audi,Make_BMW,Make_Bentley,...,Color_Gray,Color_Green,Color_Orange,Color_Other,Color_Purple,Color_Red,Color_Silver,Color_White,Color_Yellow,Color_nan
18660,0.016075,0.035714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10831,0.008760,0.035714,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5511,0.013426,0.035714,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3543,0.006407,0.026786,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12340,0.003128,0.017857,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9372,0.031366,0.062500,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
7291,0.000003,0.008929,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
17728,0.005551,0.035714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7293,0.001131,0.008929,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [83]:
y.mean()

35820.5281

##### 

In [115]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(2000,input_dim=1090, activation='relu'))
    model.add(Dense(2000, activation='relu'))
    model.add(Dense(1090, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(1, activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=1)
# define the grid search parameters
#batch_size = [100]
#epochs = [75]  
#param_grid = dict(batch_size=batch_size, epochs=epochs)
#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2, verbose =1)
model.fit(X_train, y_train, epochs = 75, batch_size =100, workers = -1, verbose=1, use_multiprocessing=True)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    


Epoch 1/75
16000/16000 [==============================] - 16s 973us/step - loss: 12261719.4125 - accuracy: 6.2500e-05
Epoch 2/75
16000/16000 [==============================] - 15s 946us/step - loss: 12261719.3313 - accuracy: 6.2500e-05
Epoch 3/75
16000/16000 [==============================] - 15s 955us/step - loss: 12261719.4062 - accuracy: 6.2500e-05
Epoch 4/75
16000/16000 [==============================] - 15s 965us/step - loss: 12261719.2188 - accuracy: 6.2500e-05
Epoch 5/75
16000/16000 [==============================] - 15s 957us/step - loss: 12261719.3875 - accuracy: 6.2500e-05
Epoch 6/75
16000/16000 [==============================] - 15s 941us/step - loss: 12261719.4563 - accuracy: 6.2500e-05
Epoch 7/75
16000/16000 [==============================] - 16s 971us/step - loss: 12261719.4375 - accuracy: 6.2500e-05
Epoch 8/75
16000/16000 [==============================] - 15s 958us/step - loss: 12261719.4000 - accuracy: 6.2500e-05
Epoch 9/75
16000/16000 [==============================] 

16000/16000 [==============================] - 15s 964us/step - loss: 12261719.2312 - accuracy: 6.2500e-05
Epoch 70/75
16000/16000 [==============================] - 16s 977us/step - loss: 12261719.4313 - accuracy: 6.2500e-05
Epoch 71/75
16000/16000 [==============================] - 16s 989us/step - loss: 12261719.4187 - accuracy: 6.2500e-05
Epoch 72/75
16000/16000 [==============================] - 16s 990us/step - loss: 12261719.3000 - accuracy: 6.2500e-05
Epoch 73/75
16000/16000 [==============================] - 15s 948us/step - loss: 12261719.4750 - accuracy: 6.2500e-05
Epoch 74/75
16000/16000 [==============================] - 15s 938us/step - loss: 12261719.3750 - accuracy: 6.2500e-05
Epoch 75/75
16000/16000 [==============================] - 15s 950us/step - loss: 12261719.3875 - accuracy: 6.2500e-05
Best: 0.002812 using {'batch_size': 100, 'epochs': 75}
0.002812 (0.000438) with: {'batch_size': 100, 'epochs': 75}


In [116]:
import numpy as np
eval(model,X_train, y_train)


16000/16000 [==============================] - 6s 345us/step
[ 20000  18771  21999 ...  24500 188995  29789]
[[495]
 [495]
 [495]
 ...
 [495]
 [495]
 [495]]
R2 of model: -1.70
RMSE of model: 1994176094.05
Base RMSE: 27176.366833266933
Normalized RSME (RMSE of predict/RMSE of base) : 1.64


44656.19883115193

In [107]:
eval(grid_result,X_train, y_train)


16000/16000 [==============================] - 1s 52us/step
[ 20000  18771  21999 ...  24500 188995  29789]
[[950]
 [950]
 [950]
 ...
 [950]
 [950]
 [950]]
GridSearchCV(cv=2, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000229BD32F588>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [100], 'epochs': [75]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)
R2 of model: -1.66
RMSE of model: 1962143878.80
Base RMSE: 27176.366833266933
Normalized RSME (RMSE of predict/RMSE of base) : 1.63


44296.09326796143

In [98]:
y_train

array([ 20000,  18771,  21999, ...,  24500, 188995,  29789], dtype=int64)

In [99]:
y_test

array([17900, 24990, 22998, ..., 38092,  9488, 17590], dtype=int64)